# Тестовое задание
## Парсинг диалогов
Главные задачи, которые должен выполнять скрипт:
- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [1]:
import pandas as pd

In [2]:
# данные
data = pd.read_csv('test_data.csv')
data.sample(5)

,dlg_id,line_n,role,text
459,5,122,client,Да да
237,2,73,manager,Угу
417,5,80,client,А если мы к вам будем вот смотрите мы сейчас н...
78,0,78,client,Такой простой шаг который в принципе можете де...
202,2,38,client,Как вот так сейчас сейчас я вам еще точно скаж...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


Датафрейм содержит 480 строк и 4 столбца:
- `dlg_id` - номер диалога (0-5)
- `line_n` - номер сообщения (реплики) в диалоге
- `role` - менеджер/клиент
- `text` - текст сообщения 

Каждая реплика состоит из одного предложения (без знаков препинания).

In [4]:
# пример сообщения
data.iloc[167]['text']

'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там'

### 1. Подготовка к решению задачи

Необходимо выделить следующие типы именованных сущностей:
- `PER` - имя менеджера
- `ORG` - название компании

Поскольку данных недостаточно для обучения собственной модели, были предприняты попытки использования уже предобученных моделей, имеющих поддержку русского языка (из библиотек *Stanza*, *PullEnti*, *Natasha*, *Spacy*). К сожалению, ни одна из выбранных моделей не показала удовлетворительного результата (ниже пример кода для Spacy). Возможно, это связано с тем, что модели обучались на письменных текстах (новости, медиа), где имена собственные начинаются с прописных букв.

In [5]:
# !pip install spacy
# !python -m spacy download ru_core_news_md
import spacy

nlp = spacy.load("ru_core_news_md")
doc = nlp(data.iloc[167]['text'])    
print(doc.ents)

()


### 2. Вспомогательные функции

Решение на основе nltk и pymorphy2.

In [6]:
# !pip install nltk
# !pip install pymorphy2
# nltk.download('punkt')

In [7]:
import nltk
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Функция **is_self_presentation** находит в сообщении паттерн представления менеджера

In [8]:
def is_self_presentation(message, name):    
    if (
        f'меня {name.lower()} зовут' in message.lower() or
        f'меня зовут {name.lower()}' in message.lower() or
        f'это {name.lower()}' in message.lower()
    ):
        return True
    return False

In [9]:
# проверка работоспособности функции
print(is_self_presentation('меня зовут ангелина', 'ангелина'))
text = data.iloc[167]['text']
print(is_self_presentation(text.lower(), 'ангелина'))

True
True


Функция **names_recognize** находит имя человека в сообщении и возвращает его, если это имя - имя менеджера. Названия компаний игнорирует.

In [10]:
def name_recognize(message):
    global morph
    prob_thresh = 0.2  # порог для определения слова-имени
    name_list = []     # на случай, если в сообщении не одно имя
    
    for word in nltk.word_tokenize(message.lower()):
        is_name = any(('Name' in p.tag and p.score >= prob_thresh) for p in morph.parse(word))
        if is_name:
            name_list.append(word)
            
    for name in name_list:
        if is_self_presentation(message.lower(), name):
            return name

In [11]:
# проверка работоспособности функции
print(name_recognize(data.iloc[3]['text']))
print(name_recognize(data.iloc[1]['text']))
print(name_recognize(data.iloc[251]['text']))

ангелина
None
максим


Функция **company_recognize** находит название компании в сообщении. В том числе учитываютс случаи, когда название компании состоит из двух и более слов.

In [12]:
def company_recognize(message):
    global morph
    if 'компания' not in message.lower():
        return
    result = ''
    for word in nltk.word_tokenize(message.lower()):
        # является ли слово началом названия компании
        is_company = any(f'компания {word}' in message.lower() and 'NOUN' in p.tag for p in morph.parse(word))
        # является ли слово частью названия компании
        is_part_name = any(
            ((len(result) > 0) and f'{result}{word}' in message.lower()) and 
            'NOUN' in p.tag for p in morph.parse(word)
        )
        
        if is_company or is_part_name:
            result += word + ' '
            
    return result.strip()

In [13]:
# проверка работоспособности функции
print(company_recognize(data.iloc[3]['text']))
print(company_recognize(data.iloc[1]['text']))
print(company_recognize(data.iloc[251]['text']))

диджитал бизнес
None
китобизнес


Функции **is_greeting** и **is_goodbye_message** определяют, является ли сообщение приветствием и прощанием соответственно.

In [14]:
def is_greeting(message):
    if (
        'добры' in message.lower() or         # добрый день, добрый вечер...
        'здравствуй' in message.lower() or    # здравствуй, здравствуйте
        'привет'in message.lower()            # привет, приветствую
    ):          
        return True

    return False

In [15]:
def is_goodbye_message(message):
    if (
        'свидани' in message.lower() or   # до свидания (и если вдруг ошибка в написании слова)
        'встречи' in message.lower() or   # до встречи 
        'хороше' in message.lower() or    # хорошего дня, хорошего понедельника... НО: не 'хорошо' - это не приветствие
        'добро' in message.lower()        # всего доброго, доброго дня...
    ): 
        return True
    
    return False

Функция **check_manager** определяет, выполнил ли менеджер требование (поздороваться в начале диалога и попрощаться в конце)

In [16]:
def check_manager(greeting, goodbye):
    if greeting and goodbye:
        return True
    return False

### 3. Парсинг текста

#### Составим парсер таким образом: 
- **вход**: подаются сообщения диалога в виде таблицы (датафрейма)
- **выход**: выдается словарь *dialog_info*, в котором содержится необходимая информация

Структура *dialog_info*:

{

    manager_name: *Name*           # имя менеджера
    company_name: *Company*        # название компании
    greeting_message: {
        1: '...'                   # номер и текст реплики приветствия
    }
    goodbye_message: {
        2: '...'                   # номер и текст реплики прощания
    }
    manager_self_presentation: {
        3: '...'                   # номер и текст реплики само-представления менеджера
    }
    requirement_done: True/False   # соблюдено ли требование

}

In [17]:
def parser_dialog(dialog):
    dialog_info = {}         # для записи итоговых значений
    dtable = dialog.copy()   # копия датасета
    
    dtable['name'] = dtable['text'].apply(name_recognize)        # имя менеджера, если оно появилось в сообщении
    dtable['company'] = dtable['text'].apply(company_recognize)  # название компании, если оно появилось
    dtable['greeting'] = dtable['text'].apply(is_greeting)       # является ли текст приветствием (true/false)
    dtable['goodbye'] = dtable['text'].apply(is_goodbye_message) # является ли текст прощанием (true/false)
    
    
    # извлекаем имя менеджера и компании
    dialog_info['manager_name'] = (''.join(list(dtable['name'].fillna('').unique()))).capitalize()
    dialog_info['company_name'] = (''.join(list(dtable['company'].fillna('').unique()))).capitalize()
    
    # если имена в диалоге не представлены
    if dialog_info['manager_name'] == '':
        dialog_info['manager_name'] = 'unknown'
    if dialog_info['company_name'] == '':
        dialog_info['company_name'] = 'unknown'
    
    
    # является ли реплика само-представлением менеджера
    dtable['self_present'] = dtable.apply(lambda x: is_self_presentation(x['text'], dialog_info['manager_name']), axis=1)
    
                                          
    # извлекаем приветственное сообщение
    dgreeting = dtable.query('greeting')
    dialog_info['greeting_message'] = {row['line_n']: row['text'] for index, row in dgreeting.iterrows()}
    
    # извлекаем прощальное сообщение
    dgoodbye = dtable.query('goodbye')
    dialog_info['goodbye_message'] = {row['line_n']: row['text'] for index, row in dgoodbye.iterrows()}
    
    # извлекаем реплику, где менеджер представляется
    dself = dtable.query('self_present')
    dialog_info['manager_self_presentation'] = {row['line_n']: row['text'] for index, row in dself.iterrows()}
                                          
    
    # выполнил ли менеджер поставленное условие (поздороваться и попрощаться)
    dialog_info['requirement_done'] = (dgreeting.shape[0] != 0 and dgoodbye.shape[0] != 0)
    
    return dialog_info

In [18]:
# отображение информации о диалоге
def display_info(dialog_info, n):
    print(f'Диалог №{n}')
    print()
    for key, value in dialog_info.items():
        if type(value) != dict:
            print(key + ':', value)
        else:
            print(key + ':')
            for key1, value1 in value.items():
                print('\t' + str(key1) + ':', value1)
            print()

И наконец, парсинг диалогов из представленного файла:

In [19]:
dialogs = {}
for i in data['dlg_id'].unique():
    dialogs[i] = parser_dialog(data.query('dlg_id == @i and role == "manager"'))

In [20]:
for dlg, info in dialogs.items():
    display_info(info, dlg)
    print('-----------', '\n\n')

Диалог №0

manager_name: Ангелина
company_name: Диджитал бизнес
greeting_message:
	1: Алло здравствуйте

goodbye_message:
	108: Всего хорошего до свидания

manager_self_presentation:
	3: Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается

requirement_done: True
----------- 


Диалог №1

manager_name: Ангелина
company_name: Диджитал бизнес
greeting_message:
	1: Алло здравствуйте

goodbye_message:
	53: Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего
	54: До свидания

manager_self_presentation:
	2: Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается

requirement_done: True
----------- 


Диалог №2

manager_name: Ангелина
company_name: Диджитал бизнес
greeting_message:
	2: Алло здравствуйте

goodbye_message:

manager_self_presentation:
	3: Меня зовут ангелина компания диджитал бизнес звоню вам по